In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from PIL import Image


def extract_feature(image):
    image_data = cv2.imread(image)
    img_gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    sift_keypoints = sift.detect(image_data, None)
    sift_g_keypoints = sift.detect(img_gray, None)
    img_keys = cv2.drawKeypoints(image_data, sift_keypoints, None)
    img_gray_keys = cv2.drawKeypoints(img_gray, sift_g_keypoints, None)
    img_keys = cv2.resize(img_keys, (224, 224))
    img_gray_keys = cv2.resize(img_gray_keys, (224, 224))
    img_keys = img_keys / 255.0
    img_gray_keys = img_gray_keys / 255.0
    tensor_img = tf.convert_to_tensor(img_keys)
    tensor_img_gray = tf.convert_to_tensor(img_gray_keys)
    return tensor_img, tensor_img_gray


def load_dataset(data_dir):
    print('Loading dataset...')
    X = []
    y = []
    print('Extracting features...')
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)
            image_tensor, gray_image_tensor = extract_feature(image_path)
            X.append(image_tensor)
            X.append(gray_image_tensor)
            y.append(label)
            y.append(label)
    return np.array(X), np.array(y)


def train_model(X_train, y_train, X_val, y_val):
    print("Training model...")
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(256, activation='relu')(x)
    
    x = Dense(128, activation='relu')(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    adam_optimizer = Adam(learning_rate=0.003)
    
    output = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    
    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy', 'precision'])
    # model.fit(X_train, to_categorical(y_train), validation_data=(X_val, to_categorical(y_val)), epochs=60, batch_size=32)
    
    history = model.fit(X_train, to_categorical(y_train), validation_data=(X_val, to_categorical(y_val)), epochs=60, batch_size=32)
        
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
    
    return model

def validate_model(model, X_val, y_val):
    print("Validating model...")
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    precision = precision_score(y_val, y_pred_classes, zero_division=0)
    accuracy = accuracy_score(y_val, y_pred_classes)
    recall = recall_score(y_val, y_pred_classes)
    return precision, accuracy, recall

def test_model(model, X_test, y_test):
    print("Testing model...")
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    precision = precision_score(y_test, y_pred_classes, zero_division=0)
    accuracy = accuracy_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    
    cm = confusion_matrix(y_test, y_pred_classes)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    
    precision, recall, _ = precision_recall_curve(y_test, y_pred[:, 1])  # Assuming binary classification and considering probability of positive class

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.show()
    return precision, accuracy, recall


data_dir = r'C:\Users\adepojuh\Documents\project\ai_and _real'
X, y = load_dataset(data_dir)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print("X: ", len(X), ' y: ', len(y))
# print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = train_model(X_train, y_train, X_val, y_val)
val_precision, val_accuracy, val_recall = validate_model(model, X_val, y_val)
print(f"Validation Precision: {val_precision:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation Recall: {val_recall:.2f}")

test_precision, test_accuracy, test_recall = test_model(model, X_test, y_test)
print(f"Test Precision: {test_precision:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Recall: {test_recall:.2f}")

X:  1950  y:  1950
Epoch 1/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.6573 - loss: 0.6947 - precision: 0.6573 - val_accuracy: 0.5670 - val_loss: 1.2411 - val_precision: 0.5670
Epoch 2/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.7681 - loss: 0.4972 - precision: 0.7681 - val_accuracy: 0.5670 - val_loss: 31.1374 - val_precision: 0.5670
Epoch 3/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.8379 - loss: 0.4199 - precision: 0.8379 - val_accuracy: 0.4330 - val_loss: 1.2228 - val_precision: 0.4330
Epoch 4/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.8661 - loss: 0.3391 - precision: 0.8661 - val_accuracy: 0.4330 - val_loss: 1.1324 - val_precision: 0.4330
Epoch 5/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.9031 - loss: 0.3162 - precision: 0.9031 - val_accuracy: 0.4330 - val_loss: 1.0283 - val_precision: 0.4330
Epoch 6/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.8736 - loss: 0.3291 - precision: 0.8736 - val_accuracy: 0.4330 - 